## Config

In [10]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

def ask(prompt: str):
    return llm.invoke([HumanMessage(content=prompt)]).content




## Zero-Shot

In [2]:
# Langkah 1: Buat prompt Zero-Shot
prompt_1 = "Apa itu Prompt Engineering dalam konteks AI?"

# Langkah 2: Kirim prompt ke model
response_1 = ask(prompt_1)

# Langkah 4: Tampilkan hasil
print("Jawaban 1:\n", response_1)

Jawaban 1:
 Prompt engineering, dalam konteks AI, adalah proses **merancang, mengembangkan, dan mengoptimalkan *prompt* (perintah atau input) yang diberikan kepada model AI, terutama model bahasa besar (LLM), untuk menghasilkan respons yang diinginkan dan berkualitas tinggi.**

Sederhananya, ini seperti memberikan instruksi yang jelas dan efektif kepada model AI agar ia dapat memahami apa yang Anda inginkan dan memberikan jawaban yang sesuai.

**Mengapa Prompt Engineering Penting?**

* **Kualitas Output:** Kualitas prompt sangat memengaruhi kualitas output yang dihasilkan oleh model AI. Prompt yang dirancang dengan baik dapat menghasilkan jawaban yang lebih akurat, relevan, koheren, dan informatif.
* **Kontrol:** Prompt engineering memungkinkan Anda untuk mengendalikan arah dan gaya respons yang dihasilkan oleh model AI. Anda dapat mengatur nada, format, dan tingkat detail jawaban.
* **Efisiensi:** Dengan prompt yang tepat, Anda dapat menghindari iterasi yang berlebihan dan mendapatkan

## Few-Shot 

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [4]:
prompt = """
Ubah format tanggal berikut dari DD-MM-YYYY ke YYYY/MM/DD:  

Contoh 1:  
Input: "25-12-2023"  
Output: "2023/12/25"  

Contoh 2:  
Input: "01-01-2024"  
Output: "2024/01/01"  

Sekarang ubah ini:  
Input: "15-05-1998"  
Output:  
"""

In [5]:
response = ask(prompt)
print("Jawaban :\n", response)

Jawaban :
 1998/05/15


## Self-Consistency 


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage


# Buat list LLM dengan berbagai temperature
temperatures = [0.3, 0.5, 0.7, 0.9]
models = [
    ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GOOGLE_API_KEY,
        temperature=temp
    ) for temp in temperatures
]

# Pertanyaan
question = "Apabila di pohon ada 10 burung, ditembak 1, sisanya berapa?"

# Loop dan tampilkan hasil
for idx, (temp, llm) in enumerate(zip(temperatures, models), 1):
    response = llm.invoke([HumanMessage(content=question)]).content
    print(f"{idx}. Model dengan temperature {temp} jawab: {response}")


1. Model dengan temperature 0.3 jawab: Ini adalah teka-teki! Jawabannya bukan 9.

Kemungkinan jawabannya adalah:

*   **Tidak ada (0):** Suara tembakan akan menakutkan semua burung yang lain, sehingga mereka akan terbang menjauh.
*   **Satu:** Burung yang ditembak jatuh, dan burung lainnya tidak peduli. (Kemungkinan kecil)

Jadi, jawaban yang paling umum dan lucu adalah **0**.
2. Model dengan temperature 0.5 jawab: Ini adalah teka-teki! Jawabannya bukan 9.

Kemungkinan jawabannya adalah:

*   **Tidak ada (0):** Suara tembakan akan menakutkan semua burung yang lain, dan mereka akan terbang menjauh.
*   **Satu:** Mungkin saja burung yang ditembak adalah satu-satunya burung yang tersisa karena yang lain sudah terbang duluan.

Teka-teki ini mengandalkan pemikiran di luar kotak dan mempertimbangkan reaksi burung-burung lain terhadap suara tembakan.
3. Model dengan temperature 0.7 jawab: Ini adalah teka-teki! Jawabannya bukan 9.

Kemungkinan jawabannya adalah:

*   **Tidak ada.** Bunyi temba

## Chain Of Thought

In [29]:
def classify_email_with_cot(email_text: str):
    cot_prompt = f"""
    You are an email classifier. Think through each step before deciding on the category.

    Classify the following email into one of the categories:
    - Personal
    - Work
    - Promotion
    - Spam

    Email: "{email_text}"

    Step-by-step reasoning:
    1. What is the email about?
    2. Who might be sending this kind of email?
    3. What is the likely purpose of the email?
    4. Based on this reasoning, which category fits best?

    Answer:
    """
    return ask(cot_prompt)


In [30]:
print(classify_email_with_cot("Don't miss our Black Friday deals – up to 80% off!"))


1.  The email is about Black Friday deals and discounts.
2.  Retail companies or businesses selling products/services are likely to send this type of email.
3.  The purpose is to promote sales and attract customers to make purchases.
4.  Therefore, the most appropriate category is **Promotion**.


In [35]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

print(classify_email_with_cot(email))

1.  **What is the email about?** The email is a welcome message to new members of the Siloam AI team, providing details about an introductory meeting.

2.  **Who might be sending this kind of email?** The email is sent from the recruitment team of a company named PT. BitHealth Teknologi Pintar.

3.  **What is the likely purpose of the email?** The purpose is to inform new team members about their welcome meeting and encourage their participation.

4.  **Based on this reasoning, which category fits best?** The email is clearly related to onboarding new employees at a company. Therefore, it falls under the **Work** category.

Answer: Work


## Tree of Thought

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

""" 
Susun Solusi Awal -> Evaluasi Solusi -> Pendalaman Strategi -> Ranking Solusi
"""


In [ ]:
# Langkah 1: Susun Solusi Awal
template1 = """Langkah 1:

Saya sedang menghadapi masalah berikut: "{input}"  
Dengan mempertimbangkan faktor-faktor ini: {perfect_factors}  
Bisakah kamu membantu saya menyusun tiga ide solusi yang berbeda? Buatlah masing-masing dengan pendekatan yang unik dan logis.

Jawaban:"""

prompt1 = PromptTemplate(
    input_variables=["input", "perfect_factors"],
    template=template1
)

chain1 = LLMChain(
    llm=llm,
    prompt=prompt1,
    output_key="solutions"
)

In [ ]:

# Langkah 2: Evaluasi Solusi
template2 = """Langkah 2:

Berikut adalah tiga solusi yang sudah diusulkan:  
{solutions}  

Sekarang, tolong nilai masing-masing solusi tersebut. Sebutkan:
- Kelebihan dan kekurangannya
- Kesulitan implementasi
- Tantangan yang mungkin muncul
- Peluang keberhasilan
- Tingkat keyakinan kamu terhadap solusi itu

Jawaban:"""

prompt2 = PromptTemplate(
    input_variables=["solutions"],
    template=template2
)

chain2 = LLMChain(
    llm=llm,
    prompt=prompt2,
    output_key="review"
)


In [ ]:
# Langkah 3: Pendalaman Strategi
template3 = """Langkah 3:

Mari kita gali lebih dalam. Untuk setiap solusi, uraikan:
- Strategi pelaksanaan
- Sumber daya atau kerja sama yang dibutuhkan
- Cara mengatasi tantangan yang mungkin muncul
- Kemungkinan dampak tak terduga dan cara menghadapinya

{review}

Jawaban:"""

prompt3 = PromptTemplate(
    input_variables=["review"],
    template=template3
)

chain3 = LLMChain(
    llm=llm,
    prompt=prompt3,
    output_key="deepen_thought_process"
)

In [ ]:
# Langkah 4: Ranking Solusi
template4 = """Langkah 4:

Dari hasil evaluasi dan pendalaman tadi, tolong urutkan solusi dari yang paling menjanjikan.  
Jelaskan alasan pemilihan urutan tersebut dan berikan saran akhir untuk masing-masing solusi.

{deepen_thought_process}

Jawaban:"""

prompt4 = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template=template4
)

chain4 = LLMChain(
    llm=llm,
    prompt=prompt4,
    output_key="ranked_solutions"
)

In [ ]:
# Tree of Thought Chain
tree_of_thought_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

In [ ]:
# Contoh Eksekusi
result = tree_of_thought_chain({
        "input": "mengurangi kemacetan di kota besar tanpa membangun jalan baru",
        "perfect_factors": "Kota ini sudah sangat padat penduduk, lahan terbatas, dan pendanaan pemerintah terbatas."
})

print("\n=== HASIL AKHIR (Ranking Solusi) ===\n")
print(result["ranked_solutions"])

## Automatic Prompt Engineer

In [63]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

In [ ]:
def generate_prompt_candidates(n=3):
    prompt = f"""
You are an expert prompt engineer.

Task: Create {n} different prompts to classify an email into one of the following categories:
- Personal
- Work
- Promotion
- Spam

Each prompt should guide the model to think before answering, include instructions if needed, and allow classification of complex real-world emails.

Use "{{email}}" as placeholder for the input email.

Format:
PROMPT 1: ...
PROMPT 2: ...
PROMPT 3: ...
"""
    return ask(prompt)



In [67]:
# 3. Ekstrak prompt individual dari teks yang dihasilkan
def extract_prompts(candidates_text, n=3):
    prompt_list = []
    for i in range(1, n + 1):
        start = f"PROMPT {i}:"
        end = f"PROMPT {i+1}:" if i < n else None
        section = candidates_text.split(start)[1]
        prompt = section.split(end)[0].strip() if end else section.strip()
        prompt_list.append(prompt)
    return prompt_list

In [ ]:
# 4. Evaluasi prompt terhadap email
def evaluate_prompts(prompt_list, test_email):
    results = []
    for i, prompt_template in enumerate(prompt_list, 1):
        filled_prompt = prompt_template.replace("{email}", test_email)
        print(f"\n================= PROMPT {i} =================\n{prompt_template}")
        print(f"\nFilled Prompt:\n{filled_prompt}")

        output = ask(filled_prompt)
        print(f"\n Model Output:\n{output.strip()}")
        results.append((f"PROMPT {i}", output.strip()))
    return results

In [69]:
# 5. Analisis hasil dengan voting label terbanyak
def analyze_results(results):
    labels = [r[1].split()[-1].replace(".", "") for r in results]
    most_common = Counter(labels).most_common(1)[0]
    return most_common, labels

In [ ]:
# Step A: Generate prompt candidates
candidate_text = generate_prompt_candidates(n=3)

# Step B: Parse jadi list of prompt strings
prompt_list = extract_prompts(candidate_text)

# Step C: Uji semua prompt ke email Siloam
results = evaluate_prompts(prompt_list, email)

# Step D: Ambil label terbanyak
most_common_label, all_labels = analyze_results(results)

print(f"\n Most consistent label: {most_common_label[0]} ({most_common_label[1]} votes)")


================= PROMPT 1 =================
**

First, analyze the email provided below and identify key indicators such as sender, recipient, subject line, keywords, and the overall tone. Consider the context: Is this email from a known contact? Does it relate to your professional responsibilities? Does it advertise a product or service? Does it contain suspicious links or requests? Classify the email into one of the following categories: Personal, Work, Promotion, or Spam. Justify your classification with specific reasons based on your analysis.

Email: {email}

Classification (choose one): Personal, Work, Promotion, Spam
Justification:
**

➡️ Filled Prompt:
**

First, analyze the email provided below and identify key indicators such as sender, recipient, subject line, keywords, and the overall tone. Consider the context: Is this email from a known contact? Does it relate to your professional responsibilities? Does it advertise a product or service? Does it contain suspicious links

## Program-Aided Language Models

In [ ]:
prompt = """
You are a Python expert.

Write a Python function called `solve()` that calculates the kinetic energy
of an object with mass = 75 kg and velocity = 20 m/s.

Use the formula: Ek = 0.5 * mass * velocity^2

Just output the function code.
"""

In [78]:
generated_code = ask(prompt)

print("🧠 Generated Code:\n")
print(generated_code)



🧠 Generated Code:

```python
def solve():
    """Calculates the kinetic energy of an object.

    Returns:
        float: The kinetic energy of the object in Joules.
    """
    mass = 75  # kg
    velocity = 20  # m/s
    kinetic_energy = 0.5 * mass * (velocity ** 2)
    return kinetic_energy
```


In [79]:
def solve():
    """Calculates the kinetic energy of an object.

    Returns:
        float: The kinetic energy of the object in Joules.
    """
    mass = 75  # kg
    velocity = 20  # m/s
    kinetic_energy = 0.5 * mass * (velocity ** 2)
    return kinetic_energy

In [81]:
hasil = solve()
print(f"\n✅ Kinetic Energy: {hasil} Joules")


✅ Kinetic Energy: 15000.0 Joules


## Multimodal COT

In [2]:
import base64

# Path ke gambar lokal
image_path = r"C:\Users\Fadhil Wicaksono\Documents\Training\Prompting-Langchain\Sepatu.png"

# Encode gambar jadi base64
def encode_image(filepath):
    with open(filepath, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

image_data = encode_image(image_path)


In [7]:
def ask_multimodal(text_prompt: str, image_base64: str):
    messages = [
        HumanMessage(
            content=[
                {"type": "text", "text": text_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
            ]
        )
    ]
    return llm.invoke(messages).content


In [10]:
multimodal_prompt = (
    "Bagus engga ini buat naik gunung merbabu untuk trail run kurang lebih 3000 mdpl, dan jenis sepatu apa yang digambar hokanya ? "
)

result = ask_multimodal(multimodal_prompt, image_data)
print(result)

Mari kita bahas tentang sepatu Hoka untuk trail run di Merbabu (kurang lebih 3000 mdpl):

*   **Apakah Sepatu Hoka Bagus untuk Trail Run di Merbabu?**

    Secara umum, sepatu Hoka bisa menjadi pilihan yang baik untuk trail run, terutama karena bantalan (cushioning) yang tebal dan nyaman. Namun, apakah sepatu *ini* cocok untuk Merbabu, itu tergantung pada beberapa faktor:

    *   **Jenis Medan:** Merbabu memiliki medan yang bervariasi, mulai dari tanah, bebatuan, hingga akar pohon. Sepatu dengan *grip* (cengkraman) yang baik sangat penting. Perhatikan solnya. Apakah pola *lug* (tonjolan pada sol) cukup dalam dan agresif? Jika tidak, sepatu ini mungkin kurang ideal untuk medan yang licin atau curam.
    *   **Kondisi Fisik dan Pengalaman:** Jika Anda terbiasa dengan sepatu minimalis atau memiliki kaki yang kuat, Anda mungkin tidak memerlukan bantalan setebal Hoka. Namun, jika Anda mencari kenyamanan dan perlindungan ekstra, Hoka bisa menjadi pilihan yang baik.
    *   **Kondisi Cuaca:*